In [ ]:
import numpy as np
import cupy as cp
import scipy as sp
from scipy import ndimage
from scipy.ndimage import gaussian_filter1d
from scipy import signal as sig

from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

import imagesize
import imageio
from PIL import Image

import matplotlib.pyplot as plt
# plt.style.use('ggplot')

from tqdm.notebook import tqdm, trange


import torch
# from fastai.vision import *
# from fastai.metrics import *

import os
import glob
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

DBG_SEED = 2718
np.random.seed(DBG_SEED)
torch.cuda.manual_seed_all(DBG_SEED)

from sporco import util
from sporco import linalg
from sporco import plot
plot.config_notebook_plotting()
from sporco.cupy import (cupy_enabled, np2cp, cp2np, select_device_by_load,
                         gpu_info)
from sporco.cupy.dictlrn import onlinecdl
from sporco.cupy.admm import cbpdn
from sporco.cupy import cnvrep
from sporco.cupy import linalg as cplinalg
from sporco.cupy.linalg import irfftn,rfftn
from sporco.cupy.linalg import inner

import torch as pt
print(torch.cuda.is_available())
import torchvision as tv



In [ ]:

class NoiseProfile:
    def __init__(self, seed=None, shape=None, dtype=np.float32):
        self.seed = seed
        self.shape = shape
        if seed:
            np.random.seed(seed)
        if shape:
            self.epsilon = self.sample(shape=shape, dtype=dtype)
        else:
            self.epsilon = None
        
    def sample(self, shape=None):
        raise NotImplementedError()
    
    def transform(self, X, idx, out=None):
        raise NotImplementedError()

class AdditiveWhiteNoise(NoiseProfile):
    def sample(self, shape, dtype=np.float32):
        self.epsilon = np.random.normal(self.mean, self.std, shape).astype(dtype)
        return self.epsilon
    
    def transform(self, X, idx, out=None):
        return np.add(X, self.epsilon[idx,...], out=out)
    
    def __init__(self, mean, std, shape=None, seed=None, dtype=np.float32):
        self.mean = mean
        
        assert 0.0 <= std, "Standard deviation must be non-negative."
        self.std = std
        
        super().__init__(seed=seed, shape=shape, dtype=dtype)

class ScalarRayleighNoise(NoiseProfile):
    def sample(self, shape=None, dtype=np.float32):
        if shape:
            self.shape = shape
        self.epsilon = np.random.rayleigh(scale=self.scale, size=self.shape).astype(dtype)
        return self.epsilon
    
    def transform(self, X, idx, out=None):
        return np.multiply(X, self.epsilon[idx,...], out=out)
    
    def __init__(self, scale, shape=None, seed=None, dtype=np.float32):
        assert 0.0 < scale, "Rayleigh scale parameter must be positive."
        self.scale = scale
        
        super().__init__(seed=seed, shape=shape, dtype=dtype)


class DropoutNoise(NoiseProfile):
    def sample(self, shape=None, dtype=np.bool):
        if shape is not None:
            self.shape = shape
        self.epsilon = np.random.binomial(1, self.mean, self.shape).astype(np.bool)
        return self.epsilon
    
    def transform(self, X, idx, out=None):
        return np.multiply(X, self.epsilon[idx,...], out=out)
    
    def __init__(self, mean, shape=None, seed=None, dtype=np.bool):
        assert 0.0 <= mean <= 1.0, "The mean dropout rate must be in [0,1]."
        self.mean = mean
        
        if not isinstance(dtype, (np.bool, bool)):
            dtype = np.bool
            warnings.warn("DropoutNoise does not support non-boolean types. Non-boolean type will be ignored.")
        
        super().__init__(seed=None, shape=shape, dtype=dtype)
            


In [ ]:
awn = AdditiveWhiteNoise(0, 1.0, shape=(150,150,3,500))
Z = np.random.normal(0,1,(5,5))
dn = DropoutNoise(0.5, shape=Z.shape + (5,))
Zd = dn.transform(Z, 0)
# bd = np.random.binomial(1, 0.2, Z.shape).astype(np.bool)
# Zd[bd] = 0.0
# print(Z)
# print(dn.epsilon[...,0])
# print(Zd)
# print(np.count_nonzero(Zd) / np.prod(Zd.shape))
dn.transform(Z, 0, out=Z)
# print(Z)

In [ ]:
class IntelSceneDataset(pt.utils.data.Dataset):    
    def applyNoise(self, func):
        return func()
        
    def getFolderList(self, path):
        return [fname for fname in os.listdir(path) if os.path.isdir(os.path.join(path, fname))]
    
    def getFileList(self, path):
        return [fname for fname in os.listdir(path) if os.path.isfile(os.path.join(path, fname))]
    
    def __init__(self, basepath, segment="train", transform=None):
        self.transform = transform
        self.basepath = basepath
        if segment == "train":
            segpath = os.path.join(self.basepath, "seg_train/seg_train")
            self.categories = self.getFolderList(segpath)
        elif segment == "test":
            segpath = os.path.join(self.basepath, "seg_test/seg_test")
            self.categories = self.getFolderList(segpath)
        labelpath = {label: os.path.join(segpath, label) for label in self.categories}
        imgtxt = "*.jpg"
        imgfnames = {label: glob.glob(os.path.join(path, "*.jpg")) for label,path in labelpath.items()}
        self.fnames = [fname for cfnames in imgfnames.values() for fname in cfnames]
        self.categories = list(imgfnames.keys())
        
        self.N = np.int64(sum(len(lst) for lst in imgfnames.values()))
        self.C = np.int64(3)
        self.imgsizes = (np.int64(150),np.int64(150)) # All images are consistent
        self.shape = (self.N, self.C) + self.imgsizes
        self.y = pt.zeros((self.N,), dtype=pt.int64)
        
        n = 0
        c = 0
        cumNum = 0
        for c,fnames in tqdm(enumerate(imgfnames.values()), "Loading Classes"):
            self.y[cumNum:cumNum+len(fnames)] = c
            cumNum += len(fnames)
            
    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        if pt.is_tensor(idx):
            idx = idx.tolist()
        
        fname = self.fnames[idx]
        
        if self.transform:
            img = self.transform(Image.open(fname).convert(mode="RGB"))
        else:
            img = Image.open(fname).convert(mode="RGB")
            
        sample = [img, self.y[idx]]

        return sample

class IntelNoiseSceneDataset(IntelSceneDataset):    
    def __init__(self,
                 basepath,
                 noise,
                 segment="train",
                 transform=None):
        super().__init__(basepath, segment, transform)
        self.noise = noise
        self.noise.sample(self.shape)
        
    def __getitem__(self, idx):
        if pt.is_tensor(idx):
            idx = idx.tolist()
        
        fname = self.fnames[idx]
        
        # Copy pil image into ndarray, scaled to [0,1] floats.
        pilimg = Image.open(fname).convert(mode="RGB").resize(self.imgsizes)
        npimg = np.array(pilimg, dtype=np.float32).transpose((2,0,1))
        
        truimg = npimg.copy()
        
        npimg /= 255.0
        
        
        # In place noise transformation of the ndarray.
        self.noise.transform(npimg, idx, out=npimg)
        
        np.clip(npimg, 0, 1, out=npimg)
        
        
        # Rescale to 255 intensity values.
        npimg *= 255.0
        
        
        Psig = np.linalg.norm(truimg.ravel(), ord=2)
        Pnoise = np.linalg.norm((truimg - npimg).ravel(), ord=2)
        SNRdB = 10.0 * np.log10(Psig / Pnoise)
        
        # Convert back to PIL image, cause pytorch collation hates numpy.
        pilimg = Image.fromarray(npimg.transpose((1,2,0)).astype(np.uint8))
        
        if self.transform:
            img = self.transform(pilimg)
        else:
            img = pilimg
        
        sample = [img, self.y[idx], Psig, Pnoise]

        return sample



In [ ]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
dbpath = os.path.abspath("/mnt/hd-storage/IntelImageClassification")
noise = AdditiveWhiteNoise(0.0, 0.1, dtype=np.float32)
trainNoiseDataset = IntelNoiseSceneDataset(dbpath, noise, transform=transform)
trainNoiseDataLoader = pt.utils.data.DataLoader(trainNoiseDataset,
                                           batch_size=4,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0)

In [ ]:
# images, labels, _, _ = iter(trainNoiseDataLoader).next()# print images
# img = (tv.utils.make_grid(images) / 2 + 0.5).numpy().transpose((1,2,0))
# plt.imshow(img).permute((2,3,1,0))
# print('GroundTruth: ', ' '.join('%5s' % list(trainNoiseDataset.categories)[labels[j]] for j in range(4)))

In [ ]:
dbpath = os.path.abspath("/mnt/hd-storage/IntelImageClassification")
trainDataset = IntelSceneDataset(dbpath, transform=transform)
trainDataLoader = pt.utils.data.DataLoader(trainDataset,
                                           batch_size=32,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0)

In [ ]:
# J = np.random.permutation(np.arange(ds.X.shape[-1], dtype=np.int64))
# idxTr = np.uint(0.80 * ds.X.shape[-1])
# JTr = np.sort(J[:idxTr])
# JTe = np.sort(J[idxTr:])

# Xtnsr = torch.tensor(np.moveaxis(np.moveaxis(ds.X, source=-1, destination=0), source=-1, destination=1))
# print(Xtnsr.shape)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 34 * 34, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # relu preserves dimensions
        x = self.pool(F.relu(self.conv1(x))) # in R^[N, Cout, |X|-|W|+1, |X|-|W|+1]
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, np.prod(x.shape[1:]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net().to(device, non_blocking=True, dtype=pt.float32)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# print(type(trainDataLoader))
# print(type(data))

In [ ]:
running_loss = 0.0
epochs = 10
lenDataIter = len(trainDataLoader)
for epoch in tqdm(range(epochs), desc="Epoch"):
    
    trainDataIter = iter(trainDataLoader)
    t = tqdm(enumerate(trainDataLoader, 0),
             desc="[%5d] loss: %.3f" % (0, 0.0),
             total = lenDataIter,
             leave=False)
    for i,data in t:
        inp,lbl = data[0].cuda(non_blocking=True), data[1].cuda(non_blocking=True)

        optimizer.zero_grad()

        outputs = net(inp)
        loss = criterion(outputs, lbl)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i+1) % 100 == 0:    # print every 2000 mini-batches
            t.set_description("[%5d] loss: %.3f" %
                  (i + 1, running_loss / 100))
            running_loss = 0.0

In [ ]:
pt.save(net, "simple_cnn.pt")

In [ ]:
net = pt.load("simple_cnn.pt")

In [ ]:
testDataset = IntelSceneDataset(dbpath, segment="train", transform=transform)
testDataLoader = pt.utils.data.DataLoader(testDataset,
                                           batch_size=4,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
images, labels = iter(testDataLoader).next()# print images
img = (tv.utils.make_grid(images) / 2 + 0.5).numpy().transpose((1,2,0))
plt.imshow(img)
print('GroundTruth: ', ' '.join('%5s' % list(testDataset.categories)[labels[j]] for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i,data in tqdm(enumerate(testDataLoader, 0), total=len(testDataLoader)):
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
Conf = np.zeros((len(testDataset.categories),len(testDataset.categories)), dtype=np.uint)
with torch.no_grad():
    for i,data in tqdm(enumerate(testDataLoader, 0), total=len(testDataLoader)):
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        for i in range(images.shape[0]):
            Conf[labels[i], predicted[i]] += np.uint(1)
Cprob = Conf.astype(np.float32) / np.sum(Conf, axis=0, keepdims=True).astype(np.float32)

cm = metrics.ConfusionMatrixDisplay(Conf, list(testDataset.categories)).plot()

In [ ]:
Cprob = Conf.astype(np.float32) / np.sum(Conf, axis=0, keepdims=True).astype(np.float32)
cm = metrics.ConfusionMatrixDisplay(Cprob, list(testDataset.categories)).plot()
print(np.sum(Cprob, axis=0))
print(np.sum(Cprob, axis=1))

In [ ]:
fig = plt.figure(figsize=(15,15))

plt.style.use('default')
cm.plot()
plt.show()

In [ ]:
# transformNoise = tv.transforms.Compose(
#     [tv.transforms.ToTensor(),
#      tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# noise = AdditiveWhiteNoise(0.0, 0.1, dtype=np.float32)
noise = ScalarRayleighNoise(np.sqrt(2.0 / np.pi), dtype=np.float32)
testNoiseDataset = IntelNoiseSceneDataset(dbpath, noise, segment="test", transform=transform)
# testNoiseDataset = IntelSceneDataset(dbpath, segment="train", transform=transformNoise)
testDataLoader = pt.utils.data.DataLoader(testNoiseDataset,
                                           batch_size=4,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0)

In [ ]:
# correct = 0
# total = 0
# with torch.no_grad():
#     for i,data in tqdm(enumerate(testDataLoader, 0), total=len(testDataLoader)):
#         images, labels = data[0].cuda(), data[1].cuda()
#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))

In [ ]:
CNoise = np.zeros((len(testNoiseDataset.categories),len(testNoiseDataset.categories)), dtype=np.uint)
SNR = [None]*len(testDataLoader)
with pt.no_grad():
    for i,data in tqdm(enumerate(testDataLoader, 0), total=len(testDataLoader)):
        images, labels, Psigs, Pnois = data[0].cuda(), data[1].cuda(), data[2], data[3]
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        SNR[i] = 10.0 * np.log10(np.array(Psigs) / np.array(Pnois))
        for j in range(images.shape[0]):
            CNoise[labels[j], predicted[j]] += np.uint(1)
CNoiseprob = CNoise.astype(np.float32) / np.sum(CNoise, axis=0, keepdims=True).astype(np.float32)
SNRNoise = np.concatenate(SNR)
cm = metrics.ConfusionMatrixDisplay(CNoise, list(testDataset.categories)).plot()
print("Accuracy is %0.4f" % (np.sum(np.diag(CNoise)) / np.sum(CNoise)))
print("Average SNR is %f" % (np.mean(SNRNoise)))

In [ ]:
images, labels, _, _ = iter(testDataLoader).next()# print images
img = (tv.utils.make_grid(images) / 2 + 0.5).numpy().transpose((1,2,0))
plt.imshow(img)
print('GroundTruth: ', ' '.join('%5s' % list(testNoiseDataset.categories)[labels[j]] for j in range(4)))

In [ ]:
with open("test_dict.npy", "rb") as f:
    D = np.load(f)

In [ ]:
print(D.shape)

In [ ]:

from sporco import util
from sporco import linalg
from sporco import plot
plot.config_notebook_plotting()
from sporco.cupy import (cupy_enabled, np2cp, cp2np, select_device_by_load,
                         gpu_info)
from sporco.cupy.dictlrn import onlinecdl
from sporco.cupy.admm import cbpdn
from sporco.cupy import cnvrep
from sporco.cupy import linalg as cplinalg
from sporco.cupy.linalg import irfftn,rfftn
from sporco.cupy.linalg import inner

In [ ]:
npd = 16
fltlmbd = 200.0
lmbda = 5.0e-2
ropt = cbpdn.ConvBPDN.Options({'Verbose': False,
                              'DataType': np.float32,
                              'MaxMainIter': 50,
                              'HighMemSolve': True,
                              'RelStopTol': 1e-7,
                              'NonNegCoef': True,
                              'FastSolve': True,
                              'rho': 1e3,
                              'AutoRho': {"Enabled": False}})
CDLConf = np.zeros((len(testNoiseDataset.categories),len(testNoiseDataset.categories)), dtype=np.uint)
t = tqdm(enumerate(testDataLoader, 0), desc="CDL Denoising", total=len(testDataLoader))
SNR = [None]*len(testDataLoader)
for m,data in t:
#     data = iter(testDataLoader).next()
    X,y,Psig,_ = data
    Xtrue = X.numpy().transpose((2,3,1,0))
    Xl, Xh = util.tikhonov_filter(0.5*Xtrue + 0.5, fltlmbd, npd)
    device = pt.device("cuda:0" if pt.cuda.is_available() else "cpu")
    X = pt.from_numpy(Xh).to(device, non_blocking=True)
    csc = cbpdn.ConvBPDN(D, X, lmbda, ropt)
    csc.solve()
    Xh_recon = cp2np(csc.reconstruct())
    X_recon_cpu = 2.0*(Xl + Xh_recon) - 1.0
    X_recon = pt.from_numpy(X_recon_cpu.transpose((3,2,0,1))).to(device, non_blocking=True)
    outputs = net(X_recon)
    _, predicted = pt.max(outputs, 1)
    Psig = np.linalg.norm(Xtrue.reshape(-1, len(y)), axis=0, ord=2)
    Prnois = np.array(np.linalg.norm((Xtrue - X_recon_cpu).reshape(-1,len(y)), axis=0, ord=2))
    SNR[m] = 10.0 * np.log10(Psig / Prnois)
    for i in range(images.shape[0]):
        CDLConf[y[i], predicted[i]] += np.uint(1)
    
    t.set_description("(Acc, SNR): (%0.4f,%f) " % (np.sum(np.diag(CDLConf)) / np.sum(CDLConf), np.mean(SNR[m])))
CDLSNR = np.concatenate(SNR)
CDLCprob = CDLConf.astype(np.float32) / np.sum(CDLConf, axis=0, keepdims=True).astype(np.float32)
print("Total Accuracy is %0.4f" % (np.sum(np.diag(CDLConf)) / np.sum(CDLConf)))
print("Average Reconstructed SNR %f" % (np.mean(CDLSNR)))

cm = metrics.ConfusionMatrixDisplay(CDLConf, list(testDataset.categories)).plot()
# [d.solve() for d in tqdm(D_recon.values(), desc="Solving Sparse Coding")]
# Y_recon = {k: cp2np(d.reconstruct()).squeeze() for k,d in D_recon.items()}
# DGf = {k: d.Df * rfftn(d.Y, s=None, axes=(0,1)) for k,d in D_recon.items()}
# DG = {k: irfftn(dgf, s=(150,150), axes=(0,1)) for k,dgf in DGf.items()}
# G_recon = {k: cp2np(d.getcoef()).squeeze() for k,d in D_recon.items()}

In [ ]:

print("Total Accuracy is %0.4f" % (np.sum(np.diag(CDLConf)) / np.sum(CDLConf)))
print("Average Reconstructed SNR %f" % (np.mean(CDLSNR)))
print(np.min(Xtrue))
print(np.max(Xtrue))
for m in range(X_recon_cpu.shape[-1]):
    plt.subplot(1,2,1)
    plt.imshow(np.clip(0.5*Xtrue[...,m] + 0.5, 0, 1))
    plt.subplot(1,2,2)
    plt.imshow(np.clip(X_recon_cpu[...,m],0,1))
    plt.show()

In [ ]:
print('Accuracy of the network on the 10000 test images: %d %%' %
      (100.0 * np.sum(np.diag(CDLConf)) / np.sum(CDLConf)))
CDLCprob = CDLConf.astype(np.float32) / np.sum(CDLConf, axis=1, keepdims=True).astype(np.float32)
cm = metrics.ConfusionMatrixDisplay(CDLCprob, list(testDataset.categories)).plot()

In [ ]:
cm = metrics.ConfusionMatrixDisplay(CNoiseprob, list(testDataset.categories)).plot()

In [ ]:
npd = 16
fltlmbd = 200.0
lmbda = 5.0e-2
ropt = cbpdn.ConvBPDN.Options({'Verbose': False,
                              'DataType': np.float32,
                              'MaxMainIter': 50,
                              'HighMemSolve': True,
                              'RelStopTol': 1e-7,
                              'NonNegCoef': True,
                              'FastSolve': True,
                              'rho': 1e3,
                              'AutoRho': {"Enabled": False}})
CDLCleanConf = np.zeros((len(testDataset.categories),len(testDataset.categories)), dtype=np.uint)
t = tqdm(enumerate(testDataLoader, 0), desc="CDL Denoising", total=len(testNoiseDataLoader))
for m,data in t:
#     data = iter(testDataLoader).next()
    X,y = data
    Xl, Xh = util.tikhonov_filter(X.numpy().transpose((2,3,1,0)), fltlmbd, npd)
    device = pt.device("cuda:0" if pt.cuda.is_available() else "cpu")
    X = pt.from_numpy(Xh).to(device, non_blocking=True)
    csc = cbpdn.ConvBPDN(D, X, lmbda, ropt)
    csc.solve()
    Xh_recon = cp2np(csc.reconstruct())
    X_recon = pt.from_numpy((Xl + Xh_recon).transpose((3,2,0,1))).to(device, non_blocking=True)
    outputs = net(X_recon)
    _, predicted = pt.max(outputs, 1)
    for i in range(images.shape[0]):
        CDLCleanConf[y[i], predicted[i]] += np.uint(1)
    
    t.set_description("Running Accuracy: %f" % (np.sum(np.diag(CDLConf)) / np.sum(CDLConf)))
CDLCleanprob = CDLCleanConf.astype(np.float32) / np.sum(CDLCleanConf, axis=0, keepdims=True).astype(np.float32)

cm = metrics.ConfusionMatrixDisplay(CDLCleanConf, list(testDataset.categories)).plot()
# [d.solve() for d in tqdm(D_recon.values(), desc="Solving Sparse Coding")]
# Y_recon = {k: cp2np(d.reconstruct()).squeeze() for k,d in D_recon.items()}
# DGf = {k: d.Df * rfftn(d.Y, s=None, axes=(0,1)) for k,d in D_recon.items()}
# DG = {k: irfftn(dgf, s=(150,150), axes=(0,1)) for k,dgf in DGf.items()}
# G_recon = {k: cp2np(d.getcoef()).squeeze() for k,d in D_recon.items()}

In [ ]:
del csc